First, we must import the needed modules to load and work with the data

In [1]:
import numpy as np
from scipy.io import loadmat

Load data containing user information, including their confusion matrix, and prior information on each image. 

In [28]:
true_labels = loadmat('true_labels.mat')
retired_images = loadmat('retired_images.mat')
conf_matrices = loadmat('conf_matrices.mat')
PP_matrices = loadmat('PP_matrices.mat')

Lets talk boout the conf_matrices.mat file, that stores the confusion matrices of all the users. This information (stored in a .datfile) will be a NX1 array where N is the number of users. in each row we have a cell array that contains the CXC "confusion matrix" for that user. A perfectly skilled user would only have values on the diagonal of this matrix and all off diagonal values indicate wrong answers were given to one category or another when presented with a 'G' true labelled image. To illustrate this we'll print out a users confusion matrix. 

In [30]:
print conf_matrices['conf_matrices'][0][0][0]

[[191   3   5   2   5   3   3   6   3   1   2   1   1   5   1]
 [  4 184   1   4   5   4   1   3   5   1   5   4   1   4   5]
 [  3   3 217   4   2   2   4   2   1   1   3   2   4   3   1]
 [  4   2   2 213   2   2   4   5   2   1   1   3   4   2   4]
 [  4   1   2   3 210   4   3   1   1   5   4   5   3   4   2]
 [  5   1   3   1   2 220   1   5   5   1   5   5   2   4   5]
 [  5   4   4   3   3   3 181   1   3   1   1   3   3   5   3]
 [  2   4   5   2   5   5   2 198   4   3   4   4   2   5   1]
 [  3   1   4   3   4   1   1   5 210   5   5   2   5   4   1]
 [  4   4   2   4   5   1   1   4   3 188   1   2   1   2   4]
 [  2   3   5   2   3   5   1   1   1   1 194   5   2   3   4]
 [  2   1   5   2   3   2   1   2   4   1   4 215   1   5   4]
 [  1   5   5   5   5   1   1   2   5   4   1   1 213   3   2]
 [  3   4   5 214   4   2 183 199   1   6   6 216 214   4   5]
 [  4   4   5   4   3   5   1   3   4   2   5   5   2   3 192]]


For example, the value in the first row, second column is a 3. This indicates that the user classified 3 images in class 2, when really they should be in class 1. Confusion matrices are used to evaluate users skill level, and updated as images are retired. 

Now, we will define the main function that evaluates the batches of images labeled by users. The Zooniverse server (specifically Nero https://github.com/zooniverse/nero) will continuously send data (batches) to be evaluated. 

Lets load in and print some of that data using the loadmat function we imported.

In [3]:
batch = loadmat('batch1.mat')
print batch

{'images': array([[ ([u'T'], [[14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 3, 13, 6, 13, 11, 5, 11, 5, 12]], [[18, 29, 24, 22, 30, 14, 13, 16, 1, 9, 7, 27, 23, 2, 21, 11, 12, 15, 28, 5, 25, 8, 20]], [[0.014317919728839477, 0.014317919728839477, 0.014317919728839477, 0.014317919728839477, 0.014317919728839477, 0.014317919728839477, 0.014317919728839477, 0.014317919728839477, 0.014317919728839477, 0.014317919728839477, 0.014317919728839477, 0.014317919728839477, 0.014317919728839477, 0.7995491237962473, 0.014317919728839477]], [[-1]], [[30]])],
       [ ([u'T'], [[6, 6, 6, 6, 6, 6, 6, 6, 6, 8, 1, 14, 7, 14, 3, 1, 6, 4, 8, 14, 10, 2]], [[4, 7, 26, 17, 3, 11, 27, 9, 6, 29, 10, 12, 28, 5, 15, 21, 13, 2, 30, 19, 20, 22]], [[0.019179049015379108, 0.019179049015379108, 0.019179049015379108, 0.019179049015379108, 0.019179049015379108, 0.7314933137846925, 0.019179049015379108, 0.019179049015379108, 0.019179049015379108, 0.019179049015379108, 0.019179049015379108, 0.019179049015379108

Okay, thats a lot of information. Lets breakdown the data these 'batches' actually store, and how to access certain parts.

The ID of the users, the ID of the image they classified, and 
the classification made by that user for that image are all held in the batches. Lets access them.

In [4]:
Sample_ImageID = batch['images'][0]['imageID'][0][0][0]
Sample_UserIDs = batch['images'][0]['IDs'][0][0]
Sample_classifications = batch['images'][0]['labels'][0][0]

print Sample_ImageID
print Sample_UserIDs
print Sample_classifications


30
[18 29 24 22 30 14 13 16  1  9  7 27 23  2 21 11 12 15 28  5 25  8 20]
[14 14 14 14 14 14 14 14 14 14 14 14 14 14  3 13  6 13 11  5 11  5 12]


The first number is the Image ID of a specific image.

The second array holds the user ID's that classified the image.

The third array holds the classifications of the image made by the users corresponding (one to one) to the second array.

Batches also contain an images Type, Machine Learning(ML) Posterior, and the True label:

The Type - A label (string) either 'T' or 'G' to determine if it is a ML classified label ('T') or a pre-labelled "golden" image ('G')

ML Posterior - An array (double) of a  1XC row vector where C is the number of pre-determined morphologies that the classifier has been trained on. Each column is the ML confidence (percentage) that the image belongs in one of the C classes.

True Label - (int) For images labelled 'T' this values is set to -1 but for images labelled 'G' This value indicates the "true" class that this image belongs in for the purposes of comparing a citizens classification with this true label.

Lets access these elements of the batches

In [5]:
Sample_type = batch['images'][0]['type'][0][0]
Sample_MLP = batch['images'][0]['ML_posterior'][0][0]
Sample_TrueLabel = batch['images'][0]['truelabel'][0][0][0]

print Sample_type
print Sample_MLP
print Sample_TrueLabel

T
[ 0.01431792  0.01431792  0.01431792  0.01431792  0.01431792  0.01431792
  0.01431792  0.01431792  0.01431792  0.01431792  0.01431792  0.01431792
  0.01431792  0.79954912  0.01431792]
-1


What does all this information mean?

This image has type 'T' meaning it is not a golden image and is still in testing.

From the ML_Posterior, we can see that the Machine is 79.95% sure this image is in the 14th class, and  1.43% sure it is in each other class.

A true label of -1 means this image is still testing (Similar meaning to its type)

Now that we understand the structure of each batch and how to access specific information within the batches, lets move on to evaluating image and user classifications.

To start, lets set a flat prior for each image. This means that before more information is analyzed, we are giving an equal chance that the image is in each of the 15 classes.

In [6]:
no_labels = np.histogram((true_labels['true_labels'][0]),np.unique(true_labels['true_labels'][0]))
priors = no_labels[1]/len(true_labels['true_labels'][0])


Initialize R_lim, the limit on how many people can look at an image before it is passed onto a higher skill level

In [7]:
R_lim = 23

Initialize N, the number of images in a batch

In [8]:
N = len(batch['images'])

initialize C, the number of morphologies (classes)

In [9]:
for i in range(N):
    if batch['images'][i][0][0] == 'T':
        C = len(batch['images'][i]['ML_posterior'][0][0])


Initialize varaible t. t is a CX1 column vector where C is the number of pre-determined morphologies and where each row is the predetermined certainty threshold that an image most surpass to be considered part of class C. Here all classes have the same threshold but in realty different categories will have more difficult or more relax thresholds for determination of class and therefore retirability.

In [10]:
t = .4*np.ones((C,1))

initialize a decision matrix that holds the decision for each image

In [11]:
 dec_matrix = np.zeros((1,N))

initialize a class matrix that holds the True labels of each image. (corresponds one to one to decision matrix)

In [12]:
class_matrix = np.zeros((1,N))

initialize a list to hold the pp_matrices for each images. We'll talk about the importance of pp_matrices later.

In [13]:
pp_matrices_rack = []

Now, we will loop over each image in the batch, first checking if it is a Golden image. If it is, we will extract the lables, userIDs for those labels, and the true labels for the image.

In [31]:
for i in range(N):
    if batch['images'][i]['type'][0][0] == 'G': #check if golden set image
        labels = batch['images'][i]['labels'][0][0] #take citizen labels of image
        userIDs = batch['images'][i]['IDs'][0][0] #take IDs of citizens who label image
        tlabel = batch['images'][i]['truelabel'][0][0][0] #take true label of image

      

In [33]:
        for ii in range(len(userIDs)): #iterate over user IDs of image

            indicator = 0

        for cc in range(len(conf_matrices['conf_matrices'][0])): #iterate over confusion matrices

          if userIDs[ii] == conf_matrices['conf_matrices'][cc]['userID'][0][0][0]: #if user is registered

            conf_matrix = conf_matrices['conf_matrices'][cc]['conf_matrix'][0] #take confusion matrix of citizen
            conf_matrix[tlabel-1,labels[ii]-1] += 1 #update confusion matrix
            conf_matrices['conf_matrices'][cc]['conf_matrix'][0] = conf_matrix #confusion matrix put back in stack
            indicator = 1

        if indicator == 0: #if user not registered

          dummy_matrix = np.zeros((C,C)) #create dummy matrix
          dummy_matrix[tlabel-1,labels[ii]-1] += 1 #update dummy matrix
          #conf_matrices['conf_matrices'] = np.append(conf_matrices['conf_matrices'][0], dummy_matrix) #append to confusion matrices
          #conf_matrices(end + 1).userID = IDs(ii)